# Задание 1

## SQL запрос

In [ ]:
'''
WITH RECURSIVE r AS (
   SELECT *
   FROM t1
   WHERE phone = '87778885566'

   UNION

   SELECT t1.*
   FROM t1
   JOIN r ON t1.phone = r.phone or t1.mail = r.mail or t1.id = r.id)

SELECT * FROM r;

'''

## Python

In [1]:
import pandas as pd 
#import numpy as np

data = [[1, 12345,89997776655,'test@mail.ru'],
        [2, 54321,87778885566,'two@mail.ru'],
        [3, 98765,87776664577,'three@mail'],
        [4, 66678,87778885566,'four@mail.ru'],
        [5, 34567,84547895566,'four@mail.ru'],
        [6, 34567,89087545678,'five@mail.ru'],
        [7, 12345,87776664577,'three@mail'],
        [8, 12845,87779664577,'ololo@mail']] # добавил две строки к исходным данным для новой связки

columns = ['key', 'id', 'phone', 'mail'] 
index = [0, 1, 2, 3, 4, 5, 6, 7]

df = pd.DataFrame(data,index, columns) 


In [2]:
b = []

def foo(x, y):
    global b
    b = y.query('mail in @x.mail or phone in @x.phone or id in @x.id')
    b = b.drop_duplicates()
    if list(x['key'])==list(b['key']):
        pass
    else:
        foo(x=b, y=y)
    
    
    
    
a = df[df['phone'] == 84547895566]
foo(x=a, y=df)
b

,key,id,phone,mail
1,2,54321,87778885566,two@mail.ru
3,4,66678,87778885566,four@mail.ru
4,5,34567,84547895566,four@mail.ru
5,6,34567,89087545678,five@mail.ru


## проверка на данных

### поиск по индексу строки

In [3]:
df_2 = pd.read_excel('test2.xlsx')

In [4]:
a = df_2.iloc[[3,4]]
foo(x=a, y=df_2)
b

,Unnamed: 0,key,id,phone,mail
3,3,3,118908,88761347200,imxcvfojmrowermtdyjxwgcr@mail.ru
4,4,4,948702,83294687381,jiaiulghdishvbqcfp@mail.ru
223394,223394,223394,948702,82207083204,wkfnyfgjlozfqymmbdqf@mail.ru
292601,292601,292601,118908,85846265287,iazlneeguyxnihvpkrprijutwngihrmv@mail.ru


### поиск по значениям

In [5]:
df = pd.read_excel('test.xlsx')

In [6]:
a_2 = df[df['id'].isin([22296, 35026, 54119])]
foo(x=a_2, y=df)
b

,Unnamed: 0,key,id,phone,mail
5194,5194,5194,54119,89552293548,kxjpfbq@mail.ru
8068,8068,8068,16137,86350580710,twrnepja@mail.ru
9249,9249,9249,35026,88859961198,erolvlycajfgqigosbcrmuoq@mail.ru
14670,14670,14670,80968,83951606408,lsdlhnzzjubio@mail.ru
15428,15428,15428,35026,83668041547,tqsbraqgufg@mail.ru
...,...,...,...,...,...
991995,991995,991995,99640,80161106086,xxhegxc@mail.ru
993913,993913,993913,36444,84951723131,oitxgixzvnqphbphsyuqcnygvjoiyngccxs@mail.ru
997754,997754,997754,36444,81892691334,zqdrfcnkcyaytapybdxrvoldqrhznxzxkha@mail.ru
998749,998749,998749,16137,88430584904,ginobvfrefglkhwyil@mail.ru


#  Задание 2

In [ ]:
'''DECLARE @columns NVARCHAR(MAX), @sql NVARCHAR(MAX);
SET @columns = N'';
SELECT @columns += N', p.' + QUOTENAME(num) 
  FROM (SELECT DISTINCT num
    from(SELECT cl.gender, l.*,
    	N'количество '+CONVERT(CHAR,ROW_NUMBER() over (PARTITION by client_name order by l.client_id, loan_date, loan_id))+N'договоров, оформленных в 2020' num
		from loans_table l join clients_table cl on cl.CLIENT_ID=l.CLIENT_ID
        WHERE loan_date BETWEEN '2020-01-01' and '2020-12-31') AS t) as t1;
SET @sql = N'
SELECT gender,' + STUFF(@columns, 1, 2, '') + '
FROM
(SELECT gender, cl.client_id,
    ''количество ''+CONVERT(CHAR,ROW_NUMBER() over (PARTITION by client_name order by l.client_id, loan_date, loan_id))+''договоров, оформленных в 2020'' num
from loans_table l join clients_table cl on cl.CLIENT_ID=l.CLIENT_ID
WHERE loan_date BETWEEN ''2020-01-01'' and ''2020-12-31'') AS j
PIVOT
(
  count(client_id) FOR num IN ('
  + STUFF(REPLACE(@columns, ', p.[', ',['), 1, 1, '')
  + ')
) AS p;';
EXEC sp_executesql @sql;'''